In [6]:
import tensorflow as tf
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pylab 
from cnn_block_interface import CnnBlockInterface

CIFAR_DIR = "./cifar-10-batches-py"
print (os.listdir(CIFAR_DIR))

['batches.meta', 'test_batch', 'data_batch_5', 'data_batch_2', 'data_batch_1', 'readme.html', 'data_batch_4', 'data_batch_3']


In [ ]:
def load_data(filename):
    with open(filename,'rb') as f:
        datas = pickle.load(f,encoding = "bytes")
        return datas[b'data'],datas[b'labels']

# 数据集的操作方法
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_datas = []
        all_labels = []
        for filename in filenames:
            datas,labels = load_data(filename)
            for data,label in zip(datas,labels):
                all_datas.append(data)
                all_labels.append(label)
        self._data = np.vstack(all_datas)
        self._data = self._data / 127.5 - 1
        self._label = np.hstack(all_labels)
        #print(self._data.shape,self._label.shape)
        
        # 数据集的大小
        self._length = self._label.shape[0]
        # 下标
        self._indicator = 0
        # 是否需要洗牌
        self._need_shuffle = need_shuffle
        if self._need_shuffle:
            self._shuffle_data()
    
    # 洗牌
    def _shuffle_data(self):
        p = np.random.permutation(self._length)
        self._data = self._data[p]
        self._label = self._label[p]
    
    # 从数据集中抽牌
    def next_batch(self,batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._length :
            if self._need_shuffle :
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else :
                raise Exception('have no more examples')
        if end_indicator > self._length :
            raise Exception('have no more examples')
        res_data,res_label = self._data[self._indicator:end_indicator],self._label[self._indicator:end_indicator]
        self._indicator = end_indicator
        return res_data, res_label

train_data = CifarData([os.path.join(CIFAR_DIR,'data_batch_%d') % i for i in range(1,6)],True)
test_data = CifarData([os.path.join(CIFAR_DIR,'test_batch')],False)

In [4]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32,[None, 3072])
y =   tf.placeholder(tf.int64,[None])

# (3072,1)
#w = tf.get_variable('w', [x.get_shape()[-1],1], initializer=tf.random_normal_initializer(0,1))
# (1,)
#b = tf.get_variable('b', [1],initializer = tf.constant_initializer(0.0))

# （None,3072）* (3072,1) = [None,1]
#y_ = tf.matmul(x, w) + b
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
matric_data, filter_data,h_conv1 = CnnBlockInterface.conv_layer(x,W_conv1,b_conv1,activation='ReLU',layer_param=(0,3,1,1))
h_pool1, matric_data_max_pos = CnnBlockInterface.pooling_layer(h_conv1,filter_size=2,stride=2)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)




# [None,1]
p_y_1 = tf.nn.sigmoid(y_)


# 损失函数的计算
# 把样例的标签整理成[None,1]的形式,int64
y_reshape = tf.reshape(y,(-1,1))
y_reshape_float32 = tf.cast(y_reshape,tf.float32)
# 要缩减的损失函数
loss = tf.reduce_mean(tf.square(y_reshape_float32 - p_y_1))


# 精度计算
# bool
predict = p_y_1 > 0.5
# 预测值与真实值的差值向量
correct_prediction = tf.equal(tf.cast(predict,tf.int64), y_reshape)
# 取均值，即为精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
'''
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_conv)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
 ''' 

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


"\nloss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_conv)\ncorrect_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))\naccuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))\n\nwith tf.name_scope('train_op'):\n    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)\n "

In [7]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val , acc_val,_ = sess.run(
            [loss,accuracy,train_op],
            feed_dict={
                x: batch_data,y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d,loss: %4.5f,acc: %4.5f'  \
                  % (i+1,loss_val, acc_val))
           
    all_test_acc_val = []
    for j in range(test_steps):
        test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        test_acc_val = sess.run([accuracy],feed_dict = {x: test_batch_data,y: test_batch_labels})
        print('预测的值是：',test_acc_val)
        print('实际的值是：',sess.run(y ,feed_dict = {x: test_batch_data,y: test_batch_labels}))
        '''
        one_pic_arr = np.reshape(test_batch_data, (28, 28))
        pic_matrix = np.matrix(one_pic_arr, dtype="float")
        plt.imshow(pic_matrix)
        pylab.show()'''
        all_test_acc_val.append(test_acc_val)
    test_acc = np.mean(all_test_acc_val)
    print('[Test] Step: %d, acc: %4.5f'% (i+1 ,test_acc))

[Train] Step: 500,loss: 0.29951,acc: 0.70000
[Train] Step: 1000,loss: 0.29949,acc: 0.70000
[Train] Step: 1500,loss: 0.25852,acc: 0.75000
[Train] Step: 2000,loss: 0.25064,acc: 0.75000
[Train] Step: 2500,loss: 0.19860,acc: 0.80000
[Train] Step: 3000,loss: 0.15397,acc: 0.80000
[Train] Step: 3500,loss: 0.25447,acc: 0.75000
[Train] Step: 4000,loss: 0.16969,acc: 0.80000
[Train] Step: 4500,loss: 0.15026,acc: 0.85000
[Train] Step: 5000,loss: 0.24837,acc: 0.70000
[Train] Step: 5500,loss: 0.19999,acc: 0.80000
[Train] Step: 6000,loss: 0.15224,acc: 0.85000
[Train] Step: 6500,loss: 0.29606,acc: 0.70000
[Train] Step: 7000,loss: 0.14363,acc: 0.85000
[Train] Step: 7500,loss: 0.22015,acc: 0.75000
[Train] Step: 8000,loss: 0.22902,acc: 0.75000
[Train] Step: 8500,loss: 0.05003,acc: 0.95000
[Train] Step: 9000,loss: 0.22518,acc: 0.75000
[Train] Step: 9500,loss: 0.11378,acc: 0.85000
[Train] Step: 10000,loss: 0.15630,acc: 0.85000
预测的值是： [0.95]
实际的值是： [0 1 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1]
预测的值是： [0.75]
实际的